# Hausdorff- and Chamfer Distance 

Package:

In [1]:
import os
import numpy as np
from skimage.draw import line, polygon
from scipy.spatial.distance import cdist

Variablen:

In [2]:
# Image size
image_width = 58
image_height = 43

path_folder_Algo='.../outputs/MOSES_Impacts'
path_folder_yolo='.../outputs/yolov8seg/surface_damage'
path_folder_Reference='.../outputs/standard_reference/surface_damage'
path_folder_mean_user='.../outputs/evaluation/surface_damage/mean_users_contours'


In [4]:
def hausdorff_distance(setA, setB):
    dists_AB = cdist(setA, setB)
    forward_hd = dists_AB.min(axis=1).max()
    backward_hd = dists_AB.min(axis=0).max()
    return max(forward_hd, backward_hd)

def chamfer_distance(setA, setB):
    dists = cdist(setA, setB)
    min_dists = np.min(dists, axis=1)
    return np.mean(min_dists), np.std(min_dists)

def chamfer_distance_symmetric(setA, setB):
    dists = cdist(setA, setB) 
    # Average of the smallest distances from S1 to S2
    term_1 = np.mean(np.min(dists, axis=1))
    # Average of the smallest distances from S2 to S1
    term_2 = np.mean(np.min(dists, axis=0))
    return term_1 + term_2

def load_yolo_contour(file_path, image_width, image_height):
    with open(file_path, 'r') as f:
        line_str = f.readline().strip()
        parts = line_str.split()[1:]
        points = np.array(list(map(float, parts))).reshape(-1, 2)
        points[:, 0] *= image_width
        points[:, 1] *= image_height
    return points

def densify_contour(contour):
    dense_points = []
    for i in range(len(contour)):
        x0, y0 = np.round(contour[i]).astype(int)
        x1, y1 = np.round(contour[(i + 1) % len(contour)]).astype(int)
        rr, cc = line(y0, x0, y1, x1)
        dense_points.extend(zip(cc, rr))
    return np.unique(np.array(dense_points), axis=0)

def get_contour_area_pixels(dense_contour, image_shape):
    rr, cc = polygon(dense_contour[:, 1], dense_contour[:, 0], shape=image_shape)
    return len(rr)

In [7]:
files_Reference = sorted([f for f in os.listdir(path_folder_Reference) if f.endswith('.txt')])
files_Algo = sorted([f for f in os.listdir(path_folder_Algo) if f.endswith('.txt')])
files_Yolo = sorted([f for f in os.listdir(path_folder_yolo) if f.endswith('.txt')])
files_mean_user=sorted([f for f in os.listdir(path_folder_mean_user) if f.endswith('_mean_user_contour_yolo_Koordinaten.txt')])

assert len(files_Reference) == len(files_Algo) == len(files_Yolo), "The number of files in the folders does not match!"

mean_over_all_Hausdorff_Algo=[]
mean_over_all_Hausdorff_YOLO=[]
mean_over_all_Hausdorff_mean_user=[]
mean_over_all_Chamfer_Algo=[]
mean_over_all_Chamfer_YOLO=[]
mean_over_all_Chamfer_mean_user=[]

image_shape = (image_height, image_width)

# Loop through all image files
for i in range(len(files_Reference)):
    filename = files_Reference[i]
    print(f"\n📄 Image {i}: {filename}")

    # Load contours
    contour_Rreference = load_yolo_contour(os.path.join(path_folder_Reference, filename), image_width, image_height)
    contour_Algo = load_yolo_contour(os.path.join(path_folder_Algo, files_Algo[i]), image_width, image_height)
    contour_Yolo = load_yolo_contour(os.path.join(path_folder_yolo, files_Yolo[i]), image_width, image_height)
    contour_mean_user=load_yolo_contour(os.path.join(path_folder_mean_user, files_mean_user[i]), image_width, image_height)

    # Create dense contours
    dense_contour_Reference = densify_contour(contour_Rreference)
    dense_contour_Algo = densify_contour(contour_Algo)
    dense_contour_Yolo = densify_contour(contour_Yolo)
    dense_contour_mean_user = densify_contour(contour_mean_user)
    

    ## Area of the reference contour in pixels
    area_pixels = get_contour_area_pixels(dense_contour_Reference, image_shape)

    # Hausdorff-Distance
    hausdorff_Algo = hausdorff_distance(dense_contour_Reference, dense_contour_Algo)
    hausdorff_Yolo = hausdorff_distance(dense_contour_Reference, dense_contour_Yolo)
    hausdorff_mean_user = hausdorff_distance(dense_contour_Reference, dense_contour_mean_user)

    #chamfer_distance_symmetric
    chamfer_Algo_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_Algo)
    chamfer_YOLO_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_Yolo)
    chamfer_mean_user_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_mean_user)
    
    mean_over_all_Hausdorff_Algo.append(hausdorff_Algo)
    mean_over_all_Hausdorff_YOLO.append(hausdorff_Yolo)
    mean_over_all_Hausdorff_mean_user.append(hausdorff_mean_user)
    mean_over_all_Chamfer_Algo.append(chamfer_Algo_sym)
    mean_over_all_Chamfer_YOLO.append(chamfer_YOLO_sym)
    mean_over_all_Chamfer_mean_user.append(chamfer_mean_user_sym)
    
    # Output
    print(f"→ Hausdorff-Distance (MOSES vs Ref): {hausdorff_Algo:.3f} px")
    print(f"→ Hausdorff-Distance (YOLO vs Ref): {hausdorff_Yolo:.3f} px")
    print(f"→ Hausdorff-Distance (user_mean vs Ref): {hausdorff_mean_user:.3f} px")
    print(f"→ Chamfer-DistanceMOSES (Sym):   {chamfer_Algo_sym:.3f} px")
    print(f"→ Chamfer-Distance Yolo (Sym):   {chamfer_YOLO_sym:.3f} px")
    print(f"→ Chamfer-Distance user_mean (Sym):   {chamfer_mean_user_sym:.3f} px")
    print(f"→ Area:   {area_pixels:.3f} px")
print('')
print('')
print(f"→ Mean over All Hausdorff-Distance (MOSES vs Ref): {np.mean(mean_over_all_Hausdorff_Algo):.3f} px")
print(f"→ STD over All Hausdorff-Distance (MOSES vs Ref): {np.std(mean_over_all_Hausdorff_Algo):.3f} px")
print(f"→ Mean over All Hausdorff-Distance (YOLO vs Ref): {np.mean(mean_over_all_Hausdorff_YOLO):.3f} px")
print(f"→ STD over All Hausdorff-Distance (YOLO vs Ref): {np.std(mean_over_all_Hausdorff_YOLO):.3f} px")
print(f"→ Mean over All Hausdorff-Distance (mean_user vs Ref): {np.mean(mean_over_all_Hausdorff_mean_user):.3f} px")
print(f"→ STD over All Hausdorff-Distance (mean_user vs Ref): {np.std(mean_over_all_Hausdorff_mean_user):.3f} px")
print('')
print('')
print(f"→ Mean over All Chamfer-Distance sym (MOSES vs Ref): {np.mean(mean_over_all_Chamfer_Algo):.3f} px")
print(f"→ STD over All Chamfer-Distance sym (MOSESvs Ref): {np.std(mean_over_all_Chamfer_Algo):.3f} px")
print(f"→ Mean over All Chamfer-Distance sym (YOLO vs Ref): {np.mean(mean_over_all_Chamfer_YOLO):.3f} px")
print(f"→ STD over All Chamfer-Distance sym (YOLO vs Ref): {np.std(mean_over_all_Chamfer_YOLO):.3f} px")
print(f"→ Mean over All Chamfer-Distance sym (mean_user vs Ref): {np.mean(mean_over_all_Chamfer_mean_user):.3f} px")
print(f"→ STD over All Chamfer-Distance sym (mean_user vs Ref): {np.std(mean_over_all_Chamfer_mean_user):.3f} px")



📄 Image 0: 1_Resize_Sensofar2D.txt
→ Hausdorff-Distance (MOSES vs Ref): 4.000 px
→ Hausdorff-Distance (YOLO vs Ref): 3.162 px
→ Hausdorff-Distance (user_mean vs Ref): 2.828 px
→ Chamfer-DistanceMOSES (Sym):   1.521 px
→ Chamfer-Distance Yolo (Sym):   1.565 px
→ Chamfer-Distance user_mean (Sym):   1.400 px
→ Area:   755.000 px

📄 Image 1: 2_Resize_Sensofar2D.txt
→ Hausdorff-Distance (MOSES vs Ref): 10.050 px
→ Hausdorff-Distance (YOLO vs Ref): 2.000 px
→ Hausdorff-Distance (user_mean vs Ref): 2.000 px
→ Chamfer-DistanceMOSES (Sym):   4.575 px
→ Chamfer-Distance Yolo (Sym):   1.085 px
→ Chamfer-Distance user_mean (Sym):   1.076 px
→ Area:   487.000 px

📄 Image 2: 3_Resize_Sensofar2D.txt
→ Hausdorff-Distance (MOSES vs Ref): 3.162 px
→ Hausdorff-Distance (YOLO vs Ref): 2.000 px
→ Hausdorff-Distance (user_mean vs Ref): 2.000 px
→ Chamfer-DistanceMOSES (Sym):   1.421 px
→ Chamfer-Distance Yolo (Sym):   1.179 px
→ Chamfer-Distance user_mean (Sym):   1.110 px
→ Area:   868.000 px

📄 Image 3: 